Import Libraries

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

Load Images

In [2]:
transform = transforms.Compose([
    transforms.Resize((32, 128)),  
    transforms.ToTensor(),         
    transforms.Normalize((0.5,), (0.5,))  
])

dataset = datasets.ImageFolder(root='task-1_data', transform=transform)

# Split dataset into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Define the CNN Model

In [3]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.flattened_size = self._get_flattened_size()
        
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(self.flattened_size, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(256, num_classes)
        )

    def _get_flattened_size(self):
        with torch.no_grad():
            dummy_input = torch.zeros(1, 3, 32, 128)
            output = self.features(dummy_input)
            return output.view(1, -1).size(1)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

Model Miscellaneous

In [4]:
num_classes = len(dataset.classes)
model = SimpleCNN(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=13888, out_features=256, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=256, out_features=100, bias=True)
  )
)

Training Loop

In [5]:
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(loader.dataset)
    return epoch_loss

Evaluation Loop

In [6]:
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(loader.dataset)
    accuracy = correct.double() / len(loader.dataset)
    return epoch_loss, accuracy

In [7]:
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Epoch 1/10, Train Loss: 4.3860, Test Loss: 3.5957, Test Accuracy: 0.2295
Epoch 2/10, Train Loss: 2.9038, Test Loss: 1.4545, Test Accuracy: 0.6982
Epoch 3/10, Train Loss: 1.5942, Test Loss: 0.6323, Test Accuracy: 0.8746
Epoch 4/10, Train Loss: 1.0751, Test Loss: 0.4351, Test Accuracy: 0.9097
Epoch 5/10, Train Loss: 0.7951, Test Loss: 0.2857, Test Accuracy: 0.9394
Epoch 6/10, Train Loss: 0.6085, Test Loss: 0.2307, Test Accuracy: 0.9469
Epoch 7/10, Train Loss: 0.5299, Test Loss: 0.1914, Test Accuracy: 0.9607
Epoch 8/10, Train Loss: 0.4283, Test Loss: 0.1714, Test Accuracy: 0.9575
Epoch 9/10, Train Loss: 0.3749, Test Loss: 0.1231, Test Accuracy: 0.9787
Epoch 10/10, Train Loss: 0.3727, Test Loss: 0.1194, Test Accuracy: 0.9734


Save Model

In [8]:
torch.save(model.state_dict(), 'task-1.pth')
print("Model saved after all epochs.")

Model saved after all epochs.
